# 5.8 若需要處理的資料量若很多

程式不會馬上跑完，此時我們就需要**通知使用者的功能**

我們先來看要如何能夠用 Python 自動發送 Email

這一個教材會介紹兩種方式：

1. 用 Python 操作 Outlook 應用程式發送 Email
2. 用 Python 透過 SMTP 模組發送 Email


---
# 5.8.1 用 Python 發送 Gmail

1. 我們需要將Google的兩步驟驗證機制開啓：[連結](https://www.google.com/intl/zh-TW/landing/2step/#tab=how-it-works)


2. 接下來我們需要一組 Google 的應用程式密碼 (App Password)：[應用程式密碼 簡介](https://support.google.com/mail/answer/185833?hl=zh-Hant)

3. 請先連至 Google Account 的頁面：[申請連結](https://myaccount.google.com/)

![](https://drive.google.com/uc?export=download&id=1dJ9-_Jjr1sj9eFYTY4EgM6TP9afUH0TG)

![](https://drive.google.com/uc?export=download&id=1fW6XA2qAqJC2A-duZ9pZ6ymz9wWv4NPp)

4. 申請應用程式密碼

![](https://drive.google.com/uc?export=download&id=1_5NurnztXzuQ_-ejNA0HziWAGeSM5Y8b)

5. 選擇 Mail

![](https://drive.google.com/uc?export=download&id=15RkEP27tI56boLTMe7sjI-s0Kno7WI7z)

6. 選擇 Others (透過程式發送)

![](https://drive.google.com/uc?export=download&id=1cnRQkhRsA81IEAWadXtUiiCUZZ9Uuz-b)

7. 輸入你的備注

![](https://drive.google.com/uc?export=download&id=17RSbSW-6kaJZWU6OOG5brMZ944-lgcn1)

8. 記下你申請到的應用程式密碼
![](https://drive.google.com/uc?export=download&id=11G57l_IJlLQYR772Ub5A4SwUBdfVjxZy)



## 5.8.2 如何用 Python 發送 Email?

SMTP (簡單郵件傳輸協定)

```python
import smtplib
# 連上 gmail 服務
conn = smtplib.SMTP('smtp.gmail.com:587')
```

建立連線：

```python
import smtplib
# 連上 gmail 服務
conn = smtplib.SMTP('smtp.gmail.com:587')
# 開啓連線
conn.ehlo()
# 加密連線
conn.starttls()
```

接下來就可以用 Python 登入 gmail

```python
conn.login('你的 email', '你的 Google 的通行碼')
```

但是我們無法發送内文有中文字的 Email...

修改一下我們的程式碼即可：
```python
import smtplib
from email.mime.text import MIMEText

sender = '寄件者 email'
receiver = '收件者 email'
content = '你好: \n這是從 Python 發送出來的 Email, 啾咪！'

msg = MIMEText(content.encode('utf-8'), _charset='utf-8')
msg['Subject'] = 'Hello 你好'
msg['From'] = sender
msg['To'] = receiver

conn = smtplib.SMTP('smtp.gmail.com:587')
conn.ehlo()
conn.starttls()
conn.login('你的 Email', '你的 Google 應用程式密碼')
conn.sendmail(sender,
              receiver,
              msg.as_string())

conn.quit()
```

爲了精簡我們的程式碼，可以將發送 Email 的功能整理成一個函數：

```python
import smtplib
from email.mime.text import MIMEText

def send_gmail(msg, sender_mail, receiver_mail, app_pass):
    sender = sender_mail
    receiver = receiver_mail
    content = msg

    msg = MIMEText(content.encode('utf-8'), _charset='utf-8')
    msg['Subject'] = 'Hello 你好'
    msg['From'] = sender
    msg['To'] = receiver

    conn = smtplib.SMTP('smtp.gmail.com:587')
    conn.ehlo()
    conn.starttls()
    conn.login(sender_mail, app_pass)
    conn.sendmail(sender,
                receiver,
                msg.as_string())

    conn.quit()
    return

send_gmail("Hello from PyXL!", "寄件者 email", "收件者 email", "你的 Google 應用程式密碼")
```

# 5.9 用 Outlook 發送 Email（限 Windows OS）

我們見識到了用 `STMP` 發送 Email，程式碼十分複雜，撰寫十分困難。

此時有另外一種透過程式發送 Email 的方法，就是使用 Python 操作 Outlook。

接下來和各位介紹 `win32com` 模組：[連結](http://timgolden.me.uk/pywin32-docs/contents.html)

`win32com` 模組是 Windows 的 API，允許使用者透過 Python 操作 Windows OS 以及應用程式相關的功能：

```python
import win32com.client
```

In [ ]:
import win32com.client

開啓 `outlook`：

```python
outlook = win32com.client.Dispatch('outlook.application')
```

開啓 `outlook` 應用程式之後，建立一個新的 Email 物件：

```python
mail = outlook.CreateItem(0)
```

我們來設定一些參數：

```python
# 收件人的 Email
mail.To = "你的 Email"
# Email 的標題
mail.Subject = "PyXL 機器人"
# Email 的本文
mail.Body = "這是 PyXL 測試！"
```

發送 Email：

```python
mail.Send()
```

若要排版本文：

```python
mail.HTMLBody = '<h3>This is HTML Body</h3>'
```

若要附件：

```python
mail.Attachments.Add(r"C:\附件檔案路徑")
```

完整版程式碼：

```python
import win32com.client

outlook = win32com.client.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = "你的 Email"
mail.Subject = "PyXL 機器人"
mail.HTMLBody = "<h3>這是 PyXL 測試！</h3>"
mail.Attachments.Add(r"C:\附件檔案路徑")
mail.Send()
```